In [1]:
import os, sys
os.chdir('Petreanu_MEI_generation')
sys.path.append(os.getcwd())

In [2]:
import os
import logging
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy import ndimage
from tqdm.auto import tqdm
import sys
from sensorium.utility.training import read_config
from loaddata.session_info import load_sessions
from utils.imagelib import load_natural_images
from utils.tuning import mean_resp_image
from utils.explorefigs import *
from loaddata.get_data_folder import get_local_drive
import shutil
import torch


# If you want to save a subset of the data, define these manually here. All three variables have to be defined. Else, leave this blank

# session_list = np.array([['LPE10885', '2023_10_20']])
# session_list = np.array(session_list)
# folders = [os.path.join(INPUT_FOLDER, 'LPE10885')]
# files = [[folder, os.path.join(folder, '2023_10_20')]
#          for folder in folders]

# Imports
# -*- coding: utf-8 -*-
"""
This script analyzes neural and behavioral data in a multi-area calcium imaging
dataset with labeled projection neurons. The visual stimuli are natural images.
Matthijs Oude Lohuis, 2023, Champalimaud Center
Anastasia Simonoff, 2024, Bernstein Center for Computational Neuroscience Berlin
"""

# Set working directory to root of repo
current_path = os.getcwd()
# Identify if path has 'Petreanu_MEI_generation' as a folder in it
if 'Petreanu_MEI_generation' in current_path:
    # If so, set the path to the root of the repo
    current_path = current_path.split('Petreanu_MEI_generation')[0] + 'Petreanu_MEI_generation'
else:
    raise FileNotFoundError(
        f'This needs to be run somewhere from within the Petreanu_MEI_generation folder, not {current_path}')
os.chdir(current_path)
sys.path.append(current_path)

run_config = read_config('run_config.yaml') # Must be set
print(run_config)

RUN_NAME = run_config['current_vals']['RUN_NAME'] # MUST be set. Creates a subfolder in the runs folder with this name, containing data, saved models, etc. IMPORTANT: all values in this folder WILL be deleted.
RUN_FOLDER = run_config['current_vals']['RUN_FOLDER']

keep_behavioral_info = run_config['data']['keep_behavioral_info']
area_of_interest = run_config['current_vals']['data']['area_of_interest']
sessions_to_keep = run_config['data']['sessions_to_keep']
INPUT_FOLDER = run_config['data']['INPUT_FOLDER']
OUTPUT_FOLDER = f'{RUN_FOLDER}/data_preprocessed' # relative to molanalysis root folder

if run_config['ASK_FOR_CONFIRMATION']:
    input(f'RUN_NAME: {RUN_NAME}\n\nINPUT FOLDER: {INPUT_FOLDER}\n\nThis will delete all files in the {RUN_FOLDER} folder. Press Enter to continue or Ctrl+C to cancel.')
else:
    print(f'RUN_NAME: {RUN_NAME}\n\nINPUT FOLDER: {INPUT_FOLDER}\n\nThis will delete all files in the {RUN_FOLDER} folder. Automatically continuing...')

# Set up logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(pathname)s - %(message)s', handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
# Create a StreamHandler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Set the logging level for the handler
# Create a Formatter and attach it to the handler
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(pathname)s - %(message)s')
console_handler.setFormatter(formatter)
# Add the handler to the logger
logger.addHandler(console_handler)

logger.info('Current working directory: %s', os.getcwd())

# TODO: Fix this so it outputs correctly during figure generation
rmap_logger = logging.getLogger('rastermap')
rmap_logger.setLevel(logging.WARNING)
rmap_logger.addHandler(console_handler)
rmap_logger.propagate = False

# Updated by Anastasia Simonoff for her local computer, etc. This should be updated for your local computer, too.

savedir = os.path.join(get_local_drive(
), 'Users\\asimo\\Documents\\BCCN\\Lab Rotations\\Petreanu Lab\\Figures\\Images' if os.environ['USERDOMAIN'] == 'ULTINTELLIGENCE' else 'OneDrive\\PostDoc\\Figures\\Images\\')
logger.info(f'Saving figures to {savedir}')

# INPUT_FOLDER = '../sensorium/notebooks/data/IM_prezipped'
# Add Add folders two levels deep from INPUT_FOLDER into a list

# delete all files in the run folder
if os.path.exists(RUN_FOLDER):
    print(f'Deleting existing folder {RUN_FOLDER}')
    shutil.rmtree(RUN_FOLDER)
else:
    os.makedirs(RUN_FOLDER, exist_ok=True)

# Delete anything in OUTPUT_FOLDER
try:
    shutil.rmtree(OUTPUT_FOLDER)
except FileNotFoundError:
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    
# test if folders already defined 
try: 
    folders
except NameError:
    # First level
    folders = [os.path.join(INPUT_FOLDER, name) for name in os.listdir(
        INPUT_FOLDER) if os.path.isdir(os.path.join(INPUT_FOLDER, name)) and not "merged_data" in name]
    folders = [x.replace("\\", "/") for x in folders]
    # Second level
    files = [[folder, os.path.join(folder, name).replace('\\', '/')] for folder in folders for name in os.listdir(
        folder) if os.path.isdir(os.path.join(folder, name)) and not "merged_data" in name]
    # only get last value after /
    session_list = [[folder.split("/")[-1], name.split("/")[-1]]
                    for folder, name in files]

    # drop ['LPE10919', '2023_11_08'] because the data is not converted yet
    session_list = [x for x in session_list if x != ['LPE10919', '2023_11_08']]
    print(session_list)

if not isinstance(sessions_to_keep, str):
    sessions_to_keep = [tuple(x) for x in sessions_to_keep]
    session_list = [tuple(x) for x in session_list]
    session_list = [x for x in session_list if x in sessions_to_keep]
    session_list = [list(x) for x in session_list]
elif sessions_to_keep == 'all':
    pass

session_list = np.array(session_list)

# Load one session including raw data: ################################################
# example session with good responses

# Load sessions lazy: (no calciumdata, behaviordata etc.,)
sessions, nSessions = load_sessions(protocol='IM', session_list=session_list, data_folder=INPUT_FOLDER)

# Load proper data and compute average trial responses:
for ises in tqdm(range(nSessions)):    # iterate over sessions

    os.makedirs(os.path.join(OUTPUT_FOLDER, session_list[ises][0], session_list[ises][1], 'data'), exist_ok=True)

    sessions[ises].load_respmat(calciumversion='deconv', keepraw=True)

    # Save respmat
    # np.save(os.path.join(files[ises][1], 'respmat.npy'), sessions[ises].respmat)
    np.save(os.path.join(OUTPUT_FOLDER, session_list[ises][0], session_list[ises][1], 'data', 'respmat.npy'), sessions[ises].respmat)


2025-03-11 16:29:20,641 - __main__ - INFO - C:\Users\Kerem Sarikaya\AppData\Local\Temp\ipykernel_31536\1608138431.py - Current working directory: d:\Python\Anastasia\Petreanu Lab\Petreanu_MEI_generation
2025-03-11 16:29:20,641 - __main__ - INFO - C:\Users\Kerem Sarikaya\AppData\Local\Temp\ipykernel_31536\1608138431.py - Current working directory: d:\Python\Anastasia\Petreanu Lab\Petreanu_MEI_generation
2025-03-11 16:29:20,641 - __main__ - INFO - C:\Users\Kerem Sarikaya\AppData\Local\Temp\ipykernel_31536\1608138431.py - Saving figures to D:/OneDrive\PostDoc\Figures\Images\
2025-03-11 16:29:20,641 - __main__ - INFO - C:\Users\Kerem Sarikaya\AppData\Local\Temp\ipykernel_31536\1608138431.py - Saving figures to D:/OneDrive\PostDoc\Figures\Images\


{'RUN_NAME_S': ['MEI_in_vivo_validation_with_grid_mean_predictor_2', 'MEI_in_vivo_validation_without_grid_mean_predictor_2'], 'ASK_FOR_CONFIRMATION': False, 'RUN_FOLDER_OVERWRITE': None, 'model': {'init_w_mean_activity': False}, 'data': {'keep_behavioral_info': False, 'areas_of_interest': ['V1', 'PM'], 'sessions_to_keep': 'all', 'INPUT_FOLDER': 'D:/Procdata/IM', 'map_to_ground_truth': False, 'gt_session_id': 'LPE13959', 'gt_date': '2025_02_26'}, 'MEIs': {'num_meis': 75, 'num_labeled_cells': 10, 'session_id': 'LPE13998', 'session_date': '2025_03_10', 'tier': 'all', 'also_output_to_local': False, 'local_output_folder': 'D:/Bonsai/lab-leopoldo-solene-vr/workflows/MEIs/', 'shape': [68, 135], 'validate_MEIs': False, 'validation_session': [['LPE13959', '2025_02_26']], 'validation_session_input_folder': 'D:/Procdata/IM'}, 'dev': {'num_models': 5}, 'current_vals': {'RUN_NAME': 'V1_MEI_in_vivo_validation_with_grid_mean_predictor_2', 'RUN_FOLDER': 'runs/V1_MEI_in_vivo_validation_with_grid_mean_p

2025-03-11 16:29:45,008 - loaddata.session_info - INFO - d:\Python\Anastasia\Petreanu Lab\Petreanu_MEI_generation\loaddata\session_info.py - ['IM'] dataset: 10 mice, 12 sessions, 67200 trials


[['LPE09665', '2023_03_20'], ['LPE10883', '2023_10_23'], ['LPE10883', '2023_10_31'], ['LPE10884', '2023_10_12'], ['LPE10885', '2023_10_20'], ['LPE10919', '2023_11_09'], ['LPE11086', '2023_12_16'], ['LPE11086', '2024_01_09'], ['LPE11495', '2024_02_29'], ['LPE11998', '2024_05_08'], ['LPE12223', '2024_06_11'], ['LPE13998', '2025_03_10']]


2025-03-11 16:29:45,026 - loaddata.session_info - INFO - d:\Python\Anastasia\Petreanu Lab\Petreanu_MEI_generation\loaddata\session_info.py - Number of neurons in PM: 11739
2025-03-11 16:29:45,026 - loaddata.session_info - INFO - d:\Python\Anastasia\Petreanu Lab\Petreanu_MEI_generation\loaddata\session_info.py - Number of neurons in V1: 17642
2025-03-11 16:29:45,026 - loaddata.session_info - INFO - d:\Python\Anastasia\Petreanu Lab\Petreanu_MEI_generation\loaddata\session_info.py - Total number of neurons: 29381


...

Computing average response for runspeed:   0%|          | 0/5600 [00:00<?, ?it/s]

Computing average response for motion energy:   0%|          | 0/5600 [00:00<?, ?it/s]

Computing average response for pupil x position:   0%|          | 0/5600 [00:00<?, ?it/s]

Computing average response for pupil y position:   0%|          | 0/5600 [00:00<?, ?it/s]

Computing average response for pupil area:   0%|          | 0/5600 [00:00<?, ?it/s]

In [6]:

def replace_nan_with_avg(arr):
    arr = arr.copy()  # Copy the array to avoid modifying the original
    nan_indices = np.where(np.isnan(arr))[0]  # Get indices of NaN values

    for i in nan_indices:
        # Handle cases where NaN is at the start or end of the array
        if i == 0:
            arr[i] = arr[i + 1]
        elif i == len(arr) - 1:
            arr[i] = arr[i - 1]
        else:
            # Replace NaN with the average of adjacent values
            arr[i] = np.nanmean([arr[i - 1], arr[i + 1]])

    return arr

# Save behavior data in sensorium format

idx_to_delete = []

In [10]:
sess_obj.animal_id

'LPE13998'

In [11]:

for i, (sess, sess_obj) in enumerate(zip(session_list, sessions)):
    folder_base = f'{OUTPUT_FOLDER}/{sess[0]}/{sess[1]}'

    pupil_size = sess_obj.respmat_pupilarea.reshape(-1, 1)
    change_of_pupil_size = sess_obj.respmat_pupilareaderiv.reshape(-1, 1)
    locotomion_speed = sess_obj.respmat_runspeed.reshape(-1, 1)

    # Data
    folder = f'{folder_base}/data/behavior'
    os.makedirs(folder, exist_ok=True)

    if np.isnan(pupil_size).all() or np.isnan(change_of_pupil_size).all() or np.isnan(locotomion_speed).all():
        logger.warning(
            f'All values in behavior data for session {sess[0]}/{sess[1]} are NaN. Session will be removed.')
        # Drop session from list
        # session_list = np.delete(session_list, i, axis=0)
        # sessions = np.delete(sessions, i, axis=0)
        idx_to_delete.append(i)
        # Remove folder
        shutil.rmtree(folder_base)
        try:
            os.rmdir(folder_base)
        except FileNotFoundError:
            pass
        try:
            shutil.rmtree(f'{INPUT_FOLDER}/{sess[0]}/{sess[1]}/')
            if len(os.listdir(f'{INPUT_FOLDER}/{sess[0]}')) == 0:
                os.rmdir(f'{INPUT_FOLDER}/{sess[0]}/')
        except FileNotFoundError:
            pass
        continue

    behavior = np.hstack((pupil_size, change_of_pupil_size, locotomion_speed))
    if keep_behavioral_info:
        behavior = replace_nan_with_avg(behavior)
        assert not np.isnan(behavior).any(), 'There are still NaN values in the behavior data'
    else:
        behavior = np.random.default_rng().normal(size=behavior.shape)

    for i in tqdm(range(behavior.shape[0]), desc=f'Saving behavior data for session {sess[0]}/{sess[1]}'):
        np.save(f'{folder}/{i}.npy', behavior[i, :])

    # Meta
    # There is also a /stimulus_Frame, but I'm not sure what the difference is
    folder = f'{folder_base}/meta/statistics/behavior/all'
    os.makedirs(folder, exist_ok=True)

    # Mean
    mean = np.mean(behavior, axis=0)
    np.save(f'{folder}/mean.npy', mean)

    # Std
    std = np.std(behavior, axis=0)
    np.save(f'{folder}/std.npy', std)

    # Min
    min = np.min(behavior, axis=0)
    np.save(f'{folder}/min.npy', min)

    # Max
    max = np.max(behavior, axis=0)
    np.save(f'{folder}/max.npy', max)

    # Median
    median = np.median(behavior, axis=0)
    np.save(f'{folder}/median.npy', median)

    # There is also a /stimulus_Frame, but I'm not sure what the difference is
    folder = f'{folder_base}/meta/statistics/behavior/stimulus_Frame'
    os.makedirs(folder, exist_ok=True)

    # Mean
    mean = np.mean(behavior, axis=0)
    np.save(f'{folder}/mean.npy', mean)

    # Std
    std = np.std(behavior, axis=0)
    np.save(f'{folder}/std.npy', std)

    # Min
    min = np.min(behavior, axis=0)
    np.save(f'{folder}/min.npy', min)

    # Max
    max = np.max(behavior, axis=0)
    np.save(f'{folder}/max.npy', max)

    # Median
    median = np.median(behavior, axis=0)
    np.save(f'{folder}/median.npy', median)

logger.warning(f'Removing sessions idx = {idx_to_delete} from sessions lists.')
session_list = np.delete(session_list, idx_to_delete, axis=0)
sessions = np.delete(sessions, idx_to_delete, axis=0)

Saving behavior data for session LPE09665/2023_03_20:   0%|          | 0/5600 [00:00<?, ?it/s]

Saving behavior data for session LPE10883/2023_10_23:   0%|          | 0/5600 [00:00<?, ?it/s]

Saving behavior data for session LPE10883/2023_10_31:   0%|          | 0/5600 [00:00<?, ?it/s]

Saving behavior data for session LPE10884/2023_10_12:   0%|          | 0/5600 [00:00<?, ?it/s]

Saving behavior data for session LPE10885/2023_10_20:   0%|          | 0/5600 [00:00<?, ?it/s]

...

Saving behavior data for session LPE11495/2024_02_29:   0%|          | 0/5600 [00:00<?, ?it/s]

Saving behavior data for session LPE11998/2024_05_08:   0%|          | 0/5600 [00:00<?, ?it/s]

Saving behavior data for session LPE12223/2024_06_11:   0%|          | 0/5600 [00:00<?, ?it/s]

Saving behavior data for session LPE13998/2025_03_10:   0%|          | 0/5600 [00:00<?, ?it/s]

2025-03-11 16:35:35,229 - __main__ - WARNING - C:\Users\Kerem Sarikaya\AppData\Local\Temp\ipykernel_31536\3313610747.py - Removing sessions idx = [] from sessions lists.
2025-03-11 16:35:35,229 - __main__ - WARNING - C:\Users\Kerem Sarikaya\AppData\Local\Temp\ipykernel_31536\3313610747.py - Removing sessions idx = [] from sessions lists.


In [16]:
sessions[-1].celldata

,iscell,iscell_prob,skew,radius,npix_soma,npix,xloc,...,meanF_chan2,noise_level,event_rate,cell_id,layer,recombinase,session_id
0,1.0,0.726542,3.867022,5.795738,130.0,171.0,113.671875,...,24.775822,12.294749,0.031356,LPE13998_2025_03_10_0_0000,L5,non,LPE13998_2025_03_10
1,1.0,0.973829,3.422527,4.668617,100.0,124.0,460.546875,...,4.981840,16.920766,0.048776,LPE13998_2025_03_10_0_0001,L5,non,LPE13998_2025_03_10
2,1.0,0.908098,2.972464,5.387129,100.0,115.0,507.421875,...,4.348712,18.714983,0.048935,LPE13998_2025_03_10_0_0002,L5,non,LPE13998_2025_03_10
3,1.0,0.978134,4.848788,3.799625,61.0,87.0,128.906250,...,15.207923,22.702727,0.032782,LPE13998_2025_03_10_0_0003,L5,non,LPE13998_2025_03_10
4,1.0,0.836163,2.538514,5.334969,102.0,120.0,221.484375,...,5.121335,16.121289,0.073481,LPE13998_2025_03_10_0_0004,L5,non,LPE13998_2025_03_10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,1.0,0.760555,1.289780,4.393959,50.0,77.0,367.968750,...,23.315962,11.453989,0.139203,LPE13998_2025_03_10_4_0102,L5,non,LPE13998_2025_03_10
354,1.0,0.606522,0.865586,6.454663,87.0,116.0,86.718750,...,11.268409,10.780393,0.138886,LPE13998_2025_03_10_4_0103,L5,non,LPE13998_2025_03_10
355,1.0,0.923202,0.689729,5.988932,107.0,120.0,113.671875,...,219.035320,11.514083,0.133185,LPE13998_2025_03_10_4_0105,L5,non,LPE13998_2025_03_10
356,1.0,0.811561,0.832167,4.987968,83.0,91.0,198.046875,...,18.099546,9.633444,0.146013,LPE13998_2025_03_10_4_0106,L5,non,LPE13998_2025_03_10


In [17]:
sess_obj = sessions[-1]

celldata = sess_obj.celldata.copy()

# layer
# V1
celldata.loc[(celldata['roi_name'] == 'V1') & (
    celldata['depth'] < 250), 'layer'] = 'L2/3'
celldata.loc[(celldata['roi_name'] == 'V1') & (
    celldata['depth'] >= 250) & (celldata['depth'] < 350), 'layer'] = 'L4'
celldata.loc[(celldata['roi_name'] == 'V1') & (
    celldata['depth'] >= 350), 'layer'] = 'L5/6'

# PM
celldata.loc[(celldata['roi_name'] == 'PM') & (
    celldata['depth'] < 250), 'layer'] = 'L2/3'
celldata.loc[(celldata['roi_name'] == 'PM') & (
    celldata['depth'] >= 250) & (celldata['depth'] < 325), 'layer'] = 'L4'
celldata.loc[(celldata['roi_name'] == 'PM') & (
    celldata['depth'] >= 325), 'layer'] = 'L5'

In [20]:
celldata['roi_name'].unique()

array(['PM'], dtype=object)

In [8]:
for sess, sess_obj in zip(session_list, sessions):
    folder_base = f'{OUTPUT_FOLDER}/{sess[0]}/{sess[1]}'
    folder = f'{folder_base}/meta/neurons'
    os.makedirs(folder, exist_ok=True)

    celldata = sess_obj.celldata.copy()

    # layer
    # V1
    celldata.loc[(celldata['roi_name'] == 'V1') & (
        celldata['depth'] < 250), 'layer'] = 'L2/3'
    celldata.loc[(celldata['roi_name'] == 'V1') & (
        celldata['depth'] >= 250) & (celldata['depth'] < 350), 'layer'] = 'L4'
    celldata.loc[(celldata['roi_name'] == 'V1') & (
        celldata['depth'] >= 350), 'layer'] = 'L5/6'

    # PM
    celldata.loc[(celldata['roi_name'] == 'PM') & (
        celldata['depth'] < 250), 'layer'] = 'L2/3'
    celldata.loc[(celldata['roi_name'] == 'PM') & (
        celldata['depth'] >= 250) & (celldata['depth'] < 325), 'layer'] = 'L4'
    celldata.loc[(celldata['roi_name'] == 'PM') & (
        celldata['depth'] >= 325), 'layer'] = 'L5'
    
    celldata = celldata.loc[celldata['roi_name'] == area_of_interest] if area_of_interest is not None else celldata
    
    # Save celldata to obj
    sess_obj.celldata = celldata.copy()

    num_neurons = len(celldata)
    
    # layer
    np.save(f'{folder}/layer.npy',
            celldata['layer'].to_numpy(dtype='<U32'))
    
    # animal ids
    np.save(f'{folder}/animal_ids.npy',
            np.full((num_neurons, ), sess_obj.animal_id, dtype='<U32'))

    # area
    np.save(f'{folder}/area.npy',
            celldata['roi_name'].to_numpy(dtype='<U32'))

    # cell motor coordinates
    np.save(f'{folder}/cell_motor_coordinates.npy',
            celldata[['xloc', 'yloc', 'depth']].to_numpy(dtype=int))

    # scan idx
    np.save(f'{folder}/scan_idx.npy',
            np.full((num_neurons, ), 0))

    # sessions
    np.save(f'{folder}/sessions.npy',
            celldata['session_id'].to_numpy(dtype='<U32'))

    # unit ids
    np.save(f'{folder}/unit_ids.npy',
            celldata['cell_id'].to_numpy(dtype='<U32'))

[]